# Extract cell level results (modified for IPQDA)

In [ ]:
import os
import numpy as np
import bigfish
import bigfish.stack as stack
import bigfish.multistack as multistack
import bigfish.plot as plot
print("Big-FISH version: {0}".format(bigfish.__version__))

#### (IPQDA: Change values here)

In [49]:
# hard-code the paths of our input and output directories
path_input = "../data/input/DMSO"
#path_input = "../data/input/JQ1"
#path_input = "../data/input/TSA"

path_output = "../data/output/DMSO"
# path_output = "../data/output/JQ1"
# path_output = "../data/output/TSA"

In this notebook, we show examples to **extract results at the cell level**. We combine the results of the previous notebook. As a reminder, segmentation is performed in 2D, but detection can be performed in 2D or 3D. Several steps are presented:
- Presentation of the different formats expected for the segmentation and detection results.
- Detection and filtering of the transcription sites.
- Extraction of the FoV results at the cell levels.
- Cell plots.

#### (IPQDA: Change values here)

In [ ]:
#Nucleus channel (DAPI)
path = os.path.join(path_input, "DMSO_004_dapi_fov_1.tif")
# path = os.path.join(path_input, "JQ1_317_dapi_fov_1.tif")
# path = os.path.join(path_input, "TSA_615_dapi_fov_1.tif")

nuc = stack.read_image(path)
nuc_mip = stack.maximum_projection(nuc)
print("smfish channel")
print("\r shape: {0}".format(nuc.shape))
print("\r dtype: {0}".format(nuc.dtype), "\n")

#FISH channel
path = os.path.join(path_input, "DMSO_004_smfish_fov_1.tif")
# path = os.path.join(path_input, "JQ1_317_smfish_fov_1.tif")
# path = os.path.join(path_input, "TSA_615_smfish_fov_1.tif")

rna = stack.read_image(path)
rna_mip = stack.maximum_projection(rna)
print("smfish channel")
print("\r shape: {0}".format(rna.shape))
print("\r dtype: {0}".format(rna.dtype))

image_contrasted = stack.rescale(rna, channel_to_stretch=0)
image_contrasted = stack.maximum_projection(image_contrasted)

## Read segmentation and detection results

The input files need to follow a specific format:
- **Segmented cells** can be saved in a tif file. It should be a 2D array with integers (np.uint8, np.uint16 or np.int64). This array is required.
- **Segmented nuclei** can be saved in a tif file. It should be a 2D array with integers (np.uint8, np.uint16 or np.int64). This array is optional. If None, transcription sites are not identified.
- **Spots coordinates** can be saved in a npy file or a csv file. It should be a 2D array with np.int64 or np.float64 dtype. This array is optional. If None, spots are not assigned to the segmented cells.
- **Clusters coordinates** can be saved in a npy file or a csv file. It should be a 2D array with np.int64 or np.float64 dtype. This array is optional. Clusters can be later defined as foci or transcription sites. If None, clusters are not identified and assigned to the segmented cells.
- **FoV images** can be saved in a png file. It should be a 2D array with np.uint8 or np.uint16 dtype. This array is optional. If None, images of the individual cells are not returned.

#### (IPQDA: Change values here)

In [ ]:
path_cell_label = os.path.join(path_output, "DMSO_004_cell_label.tif")
# path_cell_label = os.path.join(path_output, "JQ1_317_cell_label.tif")
# path_cell_label = os.path.join(path_output, "TSA_615_cell_label.tif")

path_nuc_label = os.path.join(path_output, "DMSO_004_nuc_label.tif")
# path_nuc_label = os.path.join(path_output, "JQ1_317_nuc_label.tif")
# path_nuc_label = os.path.join(path_output, "TSA_615_nuc_label.tif")

path_spots = os.path.join(path_output, "DMSO_004_spots.csv")
# path_spots = os.path.join(path_output, "JQ1_317_spots.csv")
# path_spots = os.path.join(path_output, "TSA_615_spots.csv")

path_foci = os.path.join(path_output, "DMSO_004_clusters.csv")
# path_foci = os.path.join(path_output, "JQ1_317_clusters.csv")
# path_foci = os.path.join(path_output, "TSA_615_clusters.csv")


# segmented cells
cell_label = stack.read_image(path_cell_label)
print("segmented cells")
print("\r shape: {0}".format(cell_label.shape))
print("\r dtype: {0}".format(cell_label.dtype), "\n")

# segmented nuclei
nuc_label = stack.read_image(path_nuc_label)
print("segmented nuclei")
print("\r shape: {0}".format(nuc_label.shape))
print("\r dtype: {0}".format(nuc_label.dtype), "\n")

# detected spots
spots = stack.read_array_from_csv(path_spots, dtype=np.int64)
print("detected spots")
print("\r shape: {0}".format(spots.shape))
print("\r dtype: {0}".format(spots.dtype), "\n")

# detected foci
clusters = stack.read_array_from_csv(path_foci, dtype=np.int64)
print("detected clusters")
print("\r shape: {0}".format(clusters.shape))
print("\r dtype: {0}".format(clusters.dtype))

## Transcription sites

We can discriminate **foci** from **transcription sites** with the function `bigfish.multistack.remove_transcription_site`. A transcription site is a cluster detected inside the nucleus, when the foci define the others clusters detected in the cell.

In [ ]:
spots_no_ts, foci, ts = multistack.remove_transcription_site(spots, clusters, nuc_label, ndim=3)
print("detected spots (without transcription sites)")
print("\r shape: {0}".format(spots_no_ts.shape))
print("\r dtype: {0}".format(spots_no_ts.dtype))

More generally, we can **identify objects in specific cellular compartments** with the function `bigfish.multistack.identify_objects_in_region`.

In [ ]:
spots_in, spots_out = multistack.identify_objects_in_region(nuc_label, spots, ndim=3)
print("detected spots (inside nuclei)")
print("\r shape: {0}".format(spots_in.shape))
print("\r dtype: {0}".format(spots_in.dtype), "\n")
print("detected spots (outside nuclei)")
print("\r shape: {0}".format(spots_out.shape))
print("\r dtype: {0}".format(spots_out.dtype))

## Extract cells information

**Cell extraction** is based on cell segmentation. FoV results (segmentation and detection) are assigned to each segmented cells. The function `bigfish.multistack.extract_cell` returns a list of dictionaries, one per cell. Each dictionnary can include several keys, for example:
- *cell_mask* for the cell mask in 2D.
- *cell_coord* for the cell boundary coordinates in 2D. (optional)
- *nuc_mask* for the nucleus mask in 2D. (optional)
- *nuc_coord* for the nucleus boundary coordinates in 2D. (optional)
- *rna_coord* for the detected RNAs coordinates in 3D or 2D. (optional)
- *others_coord* for additional coordinates in 3D or 2D. (optional)
- *image* for an image of the cell in 2D. (optional)
- *others_image* for additional images in 2D. (optional)

In [ ]:
fov_results = multistack.extract_cell(
    cell_label=cell_label, 
    ndim=3, 
    nuc_label=nuc_label, 
    rna_coord=spots_no_ts, 
    others_coord={"foci": foci, "transcription_site": ts},
    image=image_contrasted,
    others_image={"dapi": nuc_mip, "smfish": rna_mip})
print("number of cells identified: {0}".format(len(fov_results)))

To visualize the results of your extraction you can plot each cell with its coordinate representation (`bigfish.plot.plot_cell`) or summarize all the results in a dataframe (`bigfish.multistack.summarize_extraction_results`).

#### (IPQDA: Figure 4 - Extract cell information)

In [ ]:
for i, cell_results in enumerate(fov_results):
    print("cell {0}".format(i))
    
    # get cell results
    cell_mask = cell_results["cell_mask"]
    cell_coord = cell_results["cell_coord"]
    nuc_mask = cell_results["nuc_mask"]
    nuc_coord = cell_results["nuc_coord"]
    rna_coord = cell_results["rna_coord"]
    foci_coord = cell_results["foci"]
    ts_coord = cell_results["transcription_site"]
    image_contrasted = cell_results["image"]
    print("\r number of rna {0}".format(len(rna_coord)))
    print("\r number of foci {0}".format(len(foci_coord)))
    print("\r number of transcription sites {0}".format(len(ts_coord)))
    
    # plot cell
    plot.plot_cell(
        ndim=3, cell_coord=cell_coord, nuc_coord=nuc_coord, 
        rna_coord=rna_coord, foci_coord=foci_coord, other_coord=ts_coord, 
        image=image_contrasted, cell_mask=cell_mask, nuc_mask=nuc_mask, 
        title="Cell {0}".format(i))

#### (IPQDA: Table 1 - Extract cell information)

In [ ]:
df = multistack.summarize_extraction_results(fov_results, ndim=3)
print("shape: {0}".format(df.shape))
df.head()

## Save cell-level results

To save the results obtained at the cell-level, two methods are possible:
- Save each element in a separate file (masks in a tif file, coordinates arrays in a npy or csv file and images in png file).
- Save everything in a **single npz file** (format specific to numpy), assigning each element with a keyword. Functions `bigfish.stack.save_cell_extracted` and `bigfish.stack.read_cell_extracted` can be used for this purpose.

In [41]:
for i, cell_results in enumerate(fov_results):
    # save results
    path = os.path.join(path_output, "results_cell_{0}.npz".format(i))
    stack.save_cell_extracted(cell_results, path)

In [ ]:
# read results for the first cell of the FoV
path = os.path.join(path_output, "results_cell_0.npz")
cell_results = stack.read_cell_extracted(path, verbose=True)
print(cell_results["rna_coord"])